# Finansielle Grunnlagsdata inkludert Rentekompensasjon

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%run ./../aargang.ipynb

## Henter data
- Utføres kun på konsernregnskapset

**OBS**  
Nå hentes rentekomp kun for kommune! Skal dette beregnes for fylkeskommune også må en anne fil hentes.  
Bytt ut  
rentekomp = pd.read_parquet("../../../Testdata/arbeidsdata/KostraRegnskap/Rentekomp_kom_2021.parquet")  
til  
rentekomp = pd.read_parquet("../../../Testdata/arbeidsdata/KostraRegnskap/Rentekomp_2021.parquet")

In [ ]:
balanse = pd.read_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/bal_konsern.parquet')
kommuner = pd.read_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/' + aar + '/komm.parquet').astype(str)
fylkeskommuner = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fykomm.parquet")
rentekomp = pd.read_parquet("../../../Testdata/arbeidsdata/KostraRegnskap/Rentekomp_kom_2021.parquet")
kommunehierarki = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kommunehierarki.parquet")
fylkeskommunehierarki= pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fylkeskommunehierarki.parquet")
manglende = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/" + aar + "/kostra0b_0d_manglende_" + aar + ".parquet")

## Kalkylerenten

In [ ]:
kalkylerente = {'periode': ['2015', '2016', '2017', '2018', '2019', '2020', '2021'],
               'kalkylerente': [0.0158, 0.0168, 0.0198, 0.0237, 0.030, 0.0139, 0.0196]}
kalkylerente = pd.DataFrame(kalkylerente)

# Selvkostregnskapet for vann, avløp og renovasjon

In [ ]:
%run ./Selvkostregnskap_for_VAR.ipynb

- Filtrerer på gyldige kommuner og fylkeskommuner

**OBS**  
Nå er det kun lagt inn filter på kommune. Dette for jeg ikke har kommet til der/om disse tallene benyttes for fykom. 
Dersom fykom skal være med åpnes regionlisten og kommuner byttes ut med regioner i filteret. 

In [ ]:
#regioner = pd.concat([kommuner, fylkeskommuner])

keys = list(kommuner.columns.values)
i1 = selvkost.set_index(keys).index
i2 = kommuner.set_index(keys).index
selvkostaar = selvkost[i1.isin(i2)]

## Ordner kalkulatoriake rentekostnader for vann, avløp og renovasjon
Henter ut variablene 
- kalkulatoriske rentekostnader, intern produksjon (C1_klakrintern)
- Kalkulatoria\ske rentekostnader, ekstern produksjon (C21_kalkrsbedr)
- Gebyrgrunnlag (F_gebyrgrlag) for renovasjon
- For årganger tidligere enn 2020 benyttes kalkulatoriske rentekostnader (C_kalkrente) i stedet for C1 og C21. 

In [ ]:
if str(aar) >= '2020':
    selvkostvar = {'kommunalsektor': ['2_a','2_a','1_v','1_v','3_r','3_r','3_r'],
                   'post': ['C1_kalkrintern','C_21_kalkrsbedr','C1_kalkrintern','C_21_kalkrsbedr','C1_kalkrintern','C_21_kalkrsbedr','F_gebyrgrlag']}
else:
    selvkostvar = {'kommunalsektor': ['2_a','2_a','1_v','1_v','3_r','3_r'],
                   'post': ['C1_kalkrintern','C_21_kalkrsbedr','C1_kalkrintern','C_21_kalkrsbedr','C_kalkrente','F_gebyrgrlag']}

selvkostvar = pd.DataFrame(selvkostvar)

- Filtrerer selvkostregnskapet basert på filteret lagt over

In [ ]:
keys = list(selvkostvar.columns.values)
i1 = selvkostaar.set_index(keys).index
i2 = selvkostvar.set_index(keys).index
selvkostregn = selvkostaar[i1.isin(i2)]

### Renovasjon
- I første steg summeres rentekostnader på intern og ekstern produksjon for sektor og kommuner

In [ ]:
kalk_avfall_start = selvkostregn[selvkostregn['kommunalsektor']=='3_r'].copy()
kalk_avfall_start=kalk_avfall_start.fillna(0)

In [ ]:
kalk_avfall_prod = kalk_avfall_start[(kalk_avfall_start['post']=='C1_kalkrintern') | (kalk_avfall_start['post']=='C21_kalkrsbedr')]
kalk_avfall_prod = kalk_avfall_prod.groupby(['kommunalsektor','region','periode'])[['belop']].sum().reset_index()
kalk_avfall_prod['post'] = 'kalkrente'

*Rentekostnadene skal kun inkluderes dersom gebyrgrunnlaget er større enn null*
- Filtrer ut gebyrgrunnlaget og slår det sammen med renten
- Detter rentekostnadene til null dersom gebyr er mindre enn null

In [ ]:
kalk_avfall_gebyr = kalk_avfall_start[(kalk_avfall_start['post']=='F_gebyrgrlag')]
avfall = pd.concat([kalk_avfall_prod, kalk_avfall_gebyr])

In [ ]:
snudd = avfall.pivot_table('belop',['region','kommunalsektor','periode'], 'post' )
snudd.reset_index(drop=False, inplace=True)
snudd['kalk_avfall'] = np.where(snudd['F_gebyrgrlag']>0,snudd['kalkrente'],0)
kalk_avfall=snudd.drop(columns={'F_gebyrgrlag','kalkrente','kommunalsektor'}).rename_axis(None,axis=1)

### Vann og avløp
- Summerer sammen intern og ekstern produksjon for vann og avløp

In [ ]:
vann_avlop = selvkostregn[selvkostregn['kommunalsektor']!='3_r'].copy()
vann_avlop = vann_avlop.fillna(0)

In [ ]:
snudd_va = vann_avlop.groupby(['kommunalsektor','region','periode'])[['belop']].sum().reset_index()
snudd_va = snudd_va.pivot_table('belop',['region','periode'], 'kommunalsektor' )
snudd_va.reset_index(drop=False, inplace=True)
snudd_va.rename(columns={'1_v':'kalk_vann','2_a':'kalk_avlop'}, inplace = True)
kalk_vann_avlop = snudd_va.rename_axis(None,axis=1)

### Samler vann, avløp og renovasjon

In [ ]:
selvkostregnskapet = pd.merge(kalk_vann_avlop, kalk_avfall, how='outer', on = ['region', 'periode'])

In [ ]:
selvkostregnskapet['kalk_rente'] = selvkostregnskapet['kalk_vann'] + selvkostregnskapet['kalk_avlop'] + selvkostregnskapet['kalk_avfall']
selvkostregnskapet=selvkostregnskapet.drop(columns={'kalk_vann', 'kalk_avlop', 'kalk_avfall'})

- Setter på kalkylerenten og lager kalkulatoriske rente vann, avløp og renovasjon

In [ ]:
rente = pd.merge(selvkostregnskapet, kalkylerente, how = 'left', on = ['periode'])
rente['kalkVAR']= rente['kalk_rente'] / rente['kalkylerente']
rente=rente.drop(columns={'kalk_rente', 'kalkylerente'})

In [ ]:
rente['kalkVAR'] = np.where(rente['kalkVAR']>0,rente['kalkVAR'],0)

In [ ]:
kalkvar = pd.melt(rente,
                  id_vars=['region','periode'],
                  value_vars=list(rente.columns[2:]),
                  var_name='kapittel',
                  value_name='belop')

### Legger til Oslo fylkeskommune

In [ ]:
#Henter ut Oslo og endrer regionskode til 0300. Legger Oslo fylkeskommune på arbeidsfilen. 
oslo=kalkvar[(kalkvar['region']=='0301')].copy()
oslo['region'] = oslo['region'].str.replace('0301', '0300')

In [ ]:
kalkylerenteVAR = pd.concat([kalkvar, oslo])

## Ordner balansen
- Filtrerer på gyldige kommuner og fylkeskommuner

**OBS**  
Nå er det kun lagt inn filter på kommune. Dette for jeg ikke har kommet til der/om disse tallene benyttes for fykom. 
Dersom fykom skal være med åpnes regionlisten og kommuner byttes ut med regioner i filteret.

In [ ]:
#regioner = pd.concat([kommuner, fylkeskommuner])

keys_bal = list(kommuner.columns.values)
i1 = balanse.set_index(keys_bal).index
i2 = kommuner.set_index(keys_bal).index
bal = balanse[i1.isin(i2)]

In [ ]:
bal_var = bal[(bal['kapittel']=='KG31') | (bal['kapittel']=='KG18')]
bal_var = bal_var.drop(columns={'regnskapsomfang'})

## Ordner rentekomp

In [ ]:
rentekomp = rentekomp.rename(columns={'rentekomp':'KG38'})

In [ ]:
rentekompensasjon = pd.melt(rentekomp,
                          id_vars=['region','periode'],
                          value_vars=list(rentekomp.columns[2:]),
                          var_name='kapittel',
                          value_name='belop')

## Slår sammen rentekomp, kalkylerenteVAR og balansevariabler

In [ ]:
samlet = pd.concat([rentekompensasjon, bal_var, kalkylerenteVAR])

## Hierarkier

### Hierarki renteeksponert gjeld

In [ ]:
hierarki = {'to': ['KG39', 'KG39', 'KG39', 'KG40', 'KG40', 'KG119', 'KG119', 'KG119', 'KG118', 'KG118'],
            'from': ['KG31', 'KG38', 'kalkVAR', 'KG39','KG18', 'KG117', 'KG38', 'kalkVAR', 'KG119', 'KG18'],
            'sign': ['+', '-', '-', '+', '-', '+', '-', '-', '+', '-']}
hierarki = pd.DataFrame(hierarki)

### Hierarki region

**OBS**  
Nå er det kun lagt inn hierarki på kommune. Dette for jeg ikke har kommet til der/om disse tallene benyttes for fykom. 
Dersom fykom skal være med åpnes det samlede hierarkiet og det for kun kommuner slettes.

In [ ]:
#regionshierarki = pd.concat([fylkeskommunehierarki, kommunehierarki])
regionshierarki = kommunehierarki

### Hierarkikjøringer
- Disse må gjøres i R (siden vi ikke har en hierarkifunksjon i Python)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%R library(SSBtools)

- Sender objektene fra python til rpy2

In [ ]:
%R -i hierarki
%R -i regionshierarki
%R -i samlet

#### Renteeksponert gjeld
- Dette steget lager KG39, KG40, KG118 og KG119

In [ ]:
%%R renteeksp_kap <- HierarchyCompute(data=samlet, valueVar = "belop", 
                                     hierarchies=list(kapittel = hierarki, periode = "rowFactor",  region = "rowFactor"), 
                                     hierarchyVarNames = c(mapsFrom = "from", mapsTo = "to", sign = "sign"), 
                                     colVar = "kapittel", rowSelect = unique(samlet[, c("periode", "region")]), 
                                     inputInOutput = TRUE)

#### Regionshierarki

In [ ]:
%%R renteeksp <- HierarchyCompute(data=renteeksp_kap, valueVar = "belop", 
                                 hierarchies=list(region = regionshierarki, periode = "rowFactor",  kapittel = "rowFactor"), 
                                 hierarchyVarNames = c(mapsFrom = "from", mapsTo = "to", sign = "sign"), 
                                 colVar = "region", rowSelect = unique(renteeksp_kap[, c("periode", "kapittel")]), 
                                 inputInOutput = TRUE)

In [ ]:
%R -o renteeksp

# Samler alle begrep til endelig fil
- Her tar jeg kun med de som benyttes på kommune. De andre må jo også med - men usikker på hvor de brukes? Altså KG118 og KG119. 
- Eller brukes de ikke? Ta trengs de heller ikke lages!

- Filtrerer kun med rentekompensasjonsvariablene.
- Rentekompensasjonsvariablene lagt i renteeksponeringshierarkiet skal kun være med for kommuner som har levert balansen (0B og 0D)
    - Filtrerer ut disse først, og filtrerer bort manglende kommuner
- Filtrerer så med resten av variablene vi skal ha med på renteekp filen
- Slår sammen de to

In [ ]:
rente_var1 = ['KG39', 'KG38', 'KG40'] #skal kun ha med kommunar som har levert balanse for disse
rente_var2 = ['KG18',  'KG31']

#Filtrerer på første variabel.filter
renteeksp_filter1 = renteeksp[renteeksp['kapittel'].isin(rente_var1)]

#Lager liste over de manglende kommunene (manglende kommuner lages i flyten for KOSTRA skjema B og D). 
#Filtrerer på listen over manglende kommuner.
manglende_liste = manglende.region.unique()
renteeksp_kom_filter = renteeksp_filter1[(~renteeksp_filter1['region'].isin(manglende_liste))]

#Filtrerer på andre variabel-filter
renteeksp_filter2 = renteeksp[renteeksp['kapittel'].isin(rente_var2)]

#Samler filene
renteeksponering = pd.concat([renteeksp_kom_filter, renteeksp_filter2])

- Filtrerer ut de resterende kapitlene som skal være med videre fra balansefilen

In [ ]:
kap = ['5900', 'KG25', 'KG14', 'KG15', 'KG28', '56', 'KG21', 'KG25', 'KG32', '40', '41', '42', '43']

bal_filtrert = balanse[(balanse['kapittel'].isin(kap))]
bal_filtrert = bal_filtrert.drop(columns={'regnskapsomfang'})

- Samler balansefilene

In [ ]:
fingr = pd.concat([renteeksponering, bal_filtrert])
fingr = fingr.round(0)

In [ ]:
fingr[fingr['region']=='1103']